In [1]:
import numpy as np
import keras.applications
from keras.applications.resnet50 import preprocess_input as p_i, decode_predictions, ResNet50
from keras.preprocessing import image
import h5py
from keras.activations import softmax, relu, sigmoid
#from keras.applications.vgg19 import VGG19
#from keras.applications.vgg19 import preprocess_input
import h5py
from PIL import Image
import gdal
from keras.optimizers import Adam, SGD
from keras.initializers import glorot_normal
import keras.backend as K
from keras.layers import Dense, Input, Conv2D, Flatten, GlobalAveragePooling2D, GlobalMaxPooling2D, ZeroPadding2D, add, AveragePooling2D, InputLayer
from keras.layers.core import Dropout, Activation
from keras.layers.convolutional import MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.models import Model, Sequential
from skimage.transform import resize
import tensorflow as tf

Using TensorFlow backend.


In [15]:
config = tf.ConfigProto(log_device_placement=True,
                                        allow_soft_placement=True)
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)


In [79]:
def create_model(input_shape, top='flatten', weights=None):
    if top not in ('flatten', 'avg', 'max'):
        raise ValueError('unexpected top layer type: %s' % top)
    BottleneckLayer = {
        'flatten': Flatten(),
        'avg': GlobalAveragePooling2D(),
        'max': GlobalMaxPooling2D()
    }[top]
    base = ResNet50(input_shape=input_shape, include_top=False, weights=weights)
    x = BottleneckLayer(base.output)
    x = Dense(1, activation='linear')(x)
    model = Model(inputs=base.inputs, outputs=x)
    return model
model = create_model((222, 222, 3), weights='imagenet')

In [103]:
inp = Input((889, 889, 3))
x = Conv2D(3, 5, strides=(4, 4))(inp)
#lay[0].output

In [104]:
lay = [l for l in model.layers]
lay[14].input

<tf.Tensor 'max_pooling2d_5/MaxPool:0' shape=(?, 56, 56, 64) dtype=float32>

In [105]:
for i in range(1,4):
    lay[i].trainable = False
    x = lay[i](x)

for i in range(4, len(lay)):
    print(i, lay[i])
    lay[i].trainable = False
    x = lay[i](lay[i].input)

4 <keras.layers.core.Activation object at 0x7f58283259b0>


AttributeError: Layer activation_197 has multiple inbound nodes, hence the notion of "layer input" is ill-defined. Use `get_input_at(node_index)` instead.

In [106]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           (None, 222, 222, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       multiple             0           input_20[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 111, 111, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 111, 111, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [ ]:
model2 = Model(input=inp, output=x)

In [35]:
s = Sequential()
s.add(Conv2D(3, 5, strides=(4, 4)))
s.build()
s.compile(optimizer='sgd', loss='mse')

In [24]:
s.fit(np.ones((1, 889, 889, 3)), np.zeros((1,222,222,3)))
print("done fitting")
s.summary()

Epoch 1/1
1/1 [==============================] - 4s 4s/step - loss: 0.4854
done fitting
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 222, 222, 3)       228       
Total params: 228
Trainable params: 228
Non-trainable params: 0
_________________________________________________________________


In [25]:
l = model.layers

In [26]:
for x in l:
    s.add(x)

ValueError: Input 0 is incompatible with layer res2a_branch1: expected axis -1 of input shape to have value 64 but got shape (None, 56, 56, 256)

In [3]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    bn_axis = 3

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = Conv2D(filters1, (1, 1), name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size,
               padding='same', name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = add([x, input_tensor])
    x = Activation('relu')(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
        strides: Strides for the first conv layer in the block.
    # Returns
        Output tensor for the block.
    Note that from stage 3,
    the first conv layer at main path is with strides=(2, 2)
    And the shortcut should have strides=(2, 2) as well
    """
    filters1, filters2, filters3 = filters
    bn_axis = 3

    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    x = Conv2D(filters1, (1, 1), strides=strides,
               name=conv_name_base + '2a')(input_tensor)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters2, kernel_size, padding='same',
               name=conv_name_base + '2b')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = Activation('relu')(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + '2c')(x)
    x = BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides,
                      name=conv_name_base + '1')(input_tensor)
    shortcut = BatchNormalization(axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = add([x, shortcut])
    x = Activation('relu')(x)
    return x


def resnet50_conv(input_shape, pooling='max'):
    img_input = Input(shape=input_shape)
    x = Conv2D(3, 5, strides=(4, 4), kernel_initializer=glorot_normal())(img_input)
    x = ZeroPadding2D(padding=(3, 3), name='pad1')(x)
    x = Conv2D(64, (7, 7), strides=(2, 2), padding='valid', name='conv1')(x)
    x = BatchNormalization(axis=3, name='bn_conv1')(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block='a', strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='b')
    x = identity_block(x, 3, [64, 64, 256], stage=2, block='c')

    x = conv_block(x, 3, [128, 128, 512], stage=3, block='a')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='b')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='c')
    x = identity_block(x, 3, [128, 128, 512], stage=3, block='d')

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block='a')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='b')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='c')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='d')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='e')
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block='f')

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block='a')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='b')
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block='c')

    x = AveragePooling2D((7, 7), name='avg_pool')(x)

    # Top is not included!
    if pooling == 'avg':
        x = GlobalAveragePooling2D()(x)
    elif pooling == 'max':
        x = GlobalMaxPooling2D()(x)
    x = Dense(1)(x)
    model = Model(img_input, x, name='resnet50')
    return model

In [65]:
model = resnet50_conv((889, 889, 3))

In [61]:
model.load_weights('resnet.hdf5', by_name=True)

In [346]:
for i in range(3, len(model.layers) - 1):
    model.layers[i].trainable = True

In [284]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_50 (InputLayer)           (None, 889, 889, 3)  0                                            
__________________________________________________________________________________________________
conv2d_54 (Conv2D)              (None, 222, 222, 3)  228         input_50[0][0]                   
__________________________________________________________________________________________________
pad1 (ZeroPadding2D)            (None, 228, 228, 3)  0           conv2d_54[0][0]                  
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 111, 111, 64) 9472        pad1[0][0]                       
__________________________________________________________________________________________________
bn_conv1 (

In [285]:
model.fit(np.ones((1, 889, 889, 3)), np.zeros(1))

RuntimeError: You must compile a model before training/testing. Use `model.compile(optimizer, loss)`.

In [40]:
X = np.load("data/X_three_889.npy")
X.shape

(393, 889, 889, 3)

In [41]:
y = np.load("data/y_density.npy")
y.shape

(393,)

In [13]:
ylog =np.log(y)

In [66]:
opt = Adam(lr=1e-7)
model.compile(optimizer=opt, loss='mse')

In [67]:
model.fit(X, ylog, epochs=40)

Epoch 1/40
393/393 [==============================] - 68s 174ms/step - loss: 28.8417
Epoch 2/40
393/393 [==============================] - 15s 38ms/step - loss: 28.5719
Epoch 3/40
393/393 [==============================] - 14s 35ms/step - loss: 28.4461
Epoch 4/40
393/393 [==============================] - 14s 36ms/step - loss: 28.4409
Epoch 5/40
393/393 [==============================] - 15s 38ms/step - loss: 27.9634
Epoch 6/40
393/393 [==============================] - 13s 34ms/step - loss: 27.8893
Epoch 7/40
393/393 [==============================] - 13s 33ms/step - loss: 27.6414
Epoch 8/40
393/393 [==============================] - 14s 36ms/step - loss: 27.5806
Epoch 9/40
393/393 [==============================] - 14s 35ms/step - loss: 27.2298
Epoch 10/40
393/393 [==============================] - 14s 36ms/step - loss: 27.0432
Epoch 11/40
393/393 [==============================] - 14s 35ms/step - loss: 26.9362
Epoch 12/40
393/393 [==============================] - 13s 34ms/step - lo

KeyboardInterrupt: 

In [58]:
model.predict(X[80:85])

array([[ 81.512375],
       [158.63737 ],
       [ 76.19589 ],
       [ 60.60947 ],
       [ 59.81061 ]], dtype=float32)

In [59]:
y[80:85]

array([54.3, 58.7, 76.5, 76.5, 49.1])

In [316]:
[l for l in model.layers if l.trainable]

In [54]:
model.layers[1].get_weights()

[array([[[[ 1.03219775e-02,  6.71688020e-02, -5.55788912e-02],
          [ 5.51744141e-02, -1.20692372e-01,  5.62602617e-02],
          [ 7.65520260e-02, -2.76394039e-01, -4.20836173e-02]],
 
         [[ 2.27552578e-01, -3.66471186e-02,  6.60724267e-02],
          [ 1.21134110e-01, -3.49197239e-02, -1.84783325e-01],
          [ 8.40676948e-02, -3.28760333e-02,  3.64549756e-02]],
 
         [[-2.21269861e-01, -8.55842382e-02, -1.41874805e-01],
          [ 2.22511441e-01, -2.93031745e-02, -1.61191329e-01],
          [-1.63554534e-01, -2.18199894e-01, -9.78768617e-02]],
 
         [[-4.85967696e-02,  3.38622974e-03, -4.86462265e-02],
          [ 8.06843042e-02, -7.61183798e-02,  1.48872286e-01],
          [-4.60197516e-02, -5.28552756e-02, -1.01324432e-02]],
 
         [[ 8.63588452e-02,  8.60898867e-02, -1.06185384e-01],
          [ 2.21093297e-02,  1.67681813e-01, -7.23917037e-02],
          [ 2.68108547e-02, -5.70842326e-02,  1.66703731e-01]]],
 
 
        [[[ 8.88648480e-02,  1.042481